In [ ]:
# import needed packages
import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt

import cartopy.crs as ccrs 
import cartopy as cart
import cartopy.feature as cfeature
from datetime import datetime, timedelta




In [ ]:
# for fieldsets
field_directory = ('/storage/shared/oceanparcels/input_data/MOi/GLO12/')
input_filename_T = ('psy4v3r1-daily_T_{year_t:04d}-{month_t:02d}-{day_t:02d}.nc')


# define function to make list of files
def create_filelist(input_directory, str ,time_start,time_end,dt, dt_name):
    time = time_start
    files = []
    while (time<=time_end):
        time_tplus = time+dt_name
        y_t =time.year
        m_t = time.month
        d_t = time.day
        y_tp = time_tplus.year
        m_tp=time_tplus.month
        d_tp=time_tplus.day
        files.append(input_directory+str.format(year_t = y_t, month_t = m_t, day_t = d_t, year_tplus = y_tp, month_tplus = m_tp, day_tplus =d_tp))
        time += dt
        
    return files

#


In [ ]:
# create list of names of files for selection period 

input_list  = create_filelist(field_directory, input_filename_T ,datetime(2024,1,1),datetime(2025,1,1),timedelta(days=1), timedelta(days=1))
# Define the depth level and lat/lon range
depth_level_index = 0 # select first level
lat_min, lat_max = -20, 20
lon_min, lon_max = -90, 20

ds_test = xr.open_dataset(input_list[0])
index_lat_min = np.argmin(np.abs(ds_test.nav_lat[:,0].values -lat_min))
index_lat_max = np.argmin(np.abs(ds_test.nav_lat[:,0].values -lat_max))
index_lon_min = np.argmin(np.abs(ds_test.nav_lon[0,:].values -lon_min))
index_lon_max = np.argmin(np.abs(ds_test.nav_lon[0,:].values -lon_max))

# Open multiple files and preprocess
def preprocess(ds):
    return ds.isel(deptht=depth_level_index, x=slice(index_lon_min,index_lon_max),y=slice(index_lat_min,index_lat_max ))


In [ ]:
ds_T = xr.open_mfdataset(input_list, combine='nested', concat_dim="time_counter",preprocess= preprocess, parallel=True)

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.add_feature(cfeature.COASTLINE,edgecolor='lightgray')
ax.add_feature(cfeature.LAND, color="lightgray")
pcm = ax.pcolormesh(ds_T.nav_lon, ds_T.nav_lat, ds_T.votemper.max(dim='time_counter')) #change to .min or .mean to calculate min or mean value over year 
cbar = fig.colorbar(pcm, ax=ax, orientation="vertical", fraction=0.03)
cbar.set_label('max temperature [$^{\\circ}$]',fontsize=18)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                    linewidth=1, color='gray', alpha=1, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 20}
gl.ylabel_style =  {'size': 20}